<a href="https://colab.research.google.com/github/Vilmo18/Languages_Models_Without_Entity_Knowledge/blob/main/QA_without_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install -q transformers datasets
!pip install --upgrade pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 28.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install -q transformers datasets


In [ ]:
!pip install flair -q

## Import dataset

In [ ]:
import random
from datasets import load_dataset
from flair.models import SequenceTagger
from flair.data import Sentence

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Load the NER model
#tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')

# Define tags to be included
included_tags = {"EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
                 "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import random
from datasets import load_dataset, DatasetDict, Dataset
from flair.models import SequenceTagger
from flair.data import Sentence
from tqdm import tqdm

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Load the NER model
tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')

# Define tags to be included
included_tags = {"EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
                 "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"}

excluded_tags = {"CARDINAL", "DATA", "MONEY", "ORDINAL", "PERCENT", "QUANTITY", "TIME"}


2024-10-04 08:39:18,927 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
# train_split.save_to_disk("train_split")
# validation_split.save_to_disk("validation_split")

Saving the dataset (0/1 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
# import pickle
# with open('entity_map_qa.pkl', 'wb') as fichier:
#     pickle.dump(entity_mapping, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
from datasets import load_dataset

dataset = load_dataset("squad")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
from datasets import load_dataset
from collections import defaultdict

tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')
included_tags = {"EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
                 "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"}



2024-10-04 08:40:57,695 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.splitter import SegtokSentenceSplitter
from datasets import load_dataset
import random
from collections import defaultdict

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Load the NER model
tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')

# Define the included tags for anonymization
included_tags = {"EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP", "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"}

# Define a sentence splitter to process the text sentence by sentence
splitter = SegtokSentenceSplitter()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2024-10-05 05:32:13,211 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
# Function to anonymize text and track entity positions with consistent mapping
def anonymize_text(text, tagger, entity_mapping, used_ids):
    sentences = splitter.split(text)  # Split the text into sentences
    anonymized_text = text
    shift = 0  # To track how much the text length changes
    replacements = []

    for sentence in sentences:
        sentence_obj = Sentence(sentence.to_plain_string())
        tagger.predict(sentence_obj)

        # Process each entity within the sentence
        for entity in sentence_obj.get_spans('ner'):
            if entity.tag in included_tags:
                original_entity = entity.text

                # If the entity has been seen before, use the same ID, otherwise generate a new one
                if original_entity in entity_mapping:
                    anon_id = entity_mapping[original_entity]
                else:
                    while True:
                        anon_id = f"ent{random.randint(1, 90000)}"
                        if anon_id not in used_ids:
                            break
                    entity_mapping[original_entity] = anon_id
                    used_ids.add(anon_id)

                # Track replacements with start and end positions
                start_idx = anonymized_text.find(original_entity, shift)  # find after shift
                if start_idx != -1:
                    end_idx = start_idx + len(original_entity)
                    replacements.append((start_idx, end_idx, anon_id))
                    shift = end_idx  # Update shift to not reprocess the same text

    # Apply replacements in reverse order (to keep indices consistent)
    for start_idx, end_idx, anon_id in sorted(replacements, reverse=True):
        anonymized_text = anonymized_text[:start_idx] + anon_id + anonymized_text[end_idx:]

    return anonymized_text, replacements

# Function to adjust answer positions after anonymization
def adjust_answer_positions(example, replacements):
    for i, answer_start in enumerate(example['answers']['answer_start']):
        for start_idx, end_idx, anon_id in replacements:
            # If the answer starts inside the entity span, set it to the start of the anonymized entity
            if start_idx <= answer_start < end_idx:
                example['answers']['answer_start'][i] = start_idx
            # Adjust positions if the answer is after the replaced entity
            elif answer_start >= end_idx:
                shift = len(anon_id) - (end_idx - start_idx)
                example['answers']['answer_start'][i] += shift

    return example

# Function to anonymize an answer
def anonymize_answer(answer_text, tagger, entity_mapping, used_ids):
    anonymized_answer, _ = anonymize_text(answer_text, tagger, entity_mapping, used_ids)
    return anonymized_answer

# Function to anonymize the entire dataset example (context, question, and answers)
def anonymize_example(example, tagger, entity_mapping, used_ids):
    # Anonymize context and get replacements
    article_text = example['context']
    anonymized_article, context_replacements = anonymize_text(article_text, tagger, entity_mapping, used_ids)

    # Adjust answer positions based on context replacements
    example = adjust_answer_positions(example, context_replacements)

    # Anonymize each answer
    for i, answer_text in enumerate(example['answers']['text']):
        anonymized_answer = anonymize_answer(answer_text, tagger, entity_mapping, used_ids)
        example['answers']['text'][i] = anonymized_answer

    # Anonymize question
    question_text = example['question']
    anonymized_question, _ = anonymize_text(question_text, tagger, entity_mapping, used_ids)

    # Update the example with anonymized context and question
    example['context'] = anonymized_article
    example['question'] = anonymized_question

    return example


In [ ]:
import random
from flair.data import Sentence
from tqdm import tqdm

# Function to anonymize text and track entity positions with consistent mapping
def anonymize_text(text, tagger, entity_mapping, used_ids):
    sentences = text.split('. ')  # Split the text into sentences
    anonymized_text = text
    shift = 0  # To track how much the text length changes
    replacements = []

    for sentence in sentences:
        sentence_obj = Sentence(sentence)
        tagger.predict(sentence_obj)

        # Process each entity within the sentence
        for entity in sentence_obj.get_spans('ner'):
            if entity.tag in included_tags:
                original_entity = entity.text

                # If the entity has been seen before, use the same ID, otherwise generate a new one
                if original_entity in entity_mapping:
                    anon_id = entity_mapping[original_entity]
                else:
                    while True:
                        anon_id = f"ent{random.randint(1, 300000)}"
                        if anon_id not in used_ids:
                            break
                    entity_mapping[original_entity] = anon_id
                    used_ids.add(anon_id)

                # Track replacements with start and end positions
                start_idx = anonymized_text.find(original_entity, shift)  # find after shift
                if start_idx != -1:
                    end_idx = start_idx + len(original_entity)
                    replacements.append((start_idx, end_idx, anon_id))
                    shift = end_idx  # Update shift to not reprocess the same text

    # Apply replacements in reverse order (to keep indices consistent)
    for start_idx, end_idx, anon_id in sorted(replacements, reverse=True):
        anonymized_text = anonymized_text[:start_idx] + anon_id + anonymized_text[end_idx:]

    return anonymized_text, replacements

# Function to adjust answer positions after anonymization
def adjust_answer_positions(example, replacements):
    answer_starts = example['answers']['answer_start']

    # Loop over each answer's start position
    for i, answer_start in enumerate(answer_starts):
        shift = 0  # This tracks the overall shift for each replacement

        # Loop over the replacements and adjust positions
        for start_idx, end_idx, anon_id in replacements:
            original_entity_len = end_idx - start_idx
            anon_entity_len = len(anon_id)

            # If the answer start is within the replaced entity, shift to the start of the anonymized entity
            if start_idx <= answer_start < end_idx:
                example['answers']['answer_start'][i] = start_idx
                break
            # If the answer start is after the replaced entity, adjust for the shift caused by the length difference
            elif answer_start >= end_idx:
                shift += anon_entity_len - original_entity_len

        # Apply the accumulated shift
        example['answers']['answer_start'][i] += shift

    return example

# Function to anonymize the question and answers using the entity mapping from the context
def anonymize_question_and_answers(question_text, answers_text, tagger, entity_mapping, used_ids):
    # Anonymize the question using the context-based mapping
    anonymized_question, _ = anonymize_text(question_text, tagger, entity_mapping, used_ids)

    # Anonymize each answer using the context-based mapping
    anonymized_answers = []
    for answer_text in answers_text:
        anonymized_answer, _ = anonymize_text(answer_text, tagger, entity_mapping, used_ids)
        anonymized_answers.append(anonymized_answer)

    return anonymized_question, anonymized_answers

# Function to anonymize the entire dataset example (context, question, and answers)
# and return the entity mapping as well
def anonymize_example_with_mapping(example, tagger):
    entity_mapping = {}  # Keep track of entity to anonymized ID mapping within one example
    used_ids = set()     # Keep track of all used anonymized IDs within the example

    # Step 1: Anonymize context and get replacements
    article_text = example['context']
    anonymized_article, context_replacements = anonymize_text(article_text, tagger, entity_mapping, used_ids)

    # Step 2: Adjust answer positions based on context replacements
    example = adjust_answer_positions(example, context_replacements)

    # Step 3: Anonymize question and answers based on the context mapping
    question_text = example['question']
    answers_text = example['answers']['text']
    anonymized_question, anonymized_answers = anonymize_question_and_answers(question_text, answers_text, tagger, entity_mapping, used_ids)

    # Step 4: Update the example with anonymized context, question, and answers
    example['context'] = anonymized_article
    example['question'] = anonymized_question
    example['answers']['text'] = anonymized_answers

    # Return the anonymized example and the entity mapping
    return example, entity_mapping

# Function to anonymize a dataset (list of examples) and return all entity mappings
def anonymize_dataset(dataset, tagger):
    all_mappings = []  # List to store mappings for each example
    anonymized_dataset = []  # Store anonymized examples

    # Process each example in the dataset
    for example in tqdm(dataset):
        anonymized_example, entity_mapping = anonymize_example_with_mapping(example, tagger)
        anonymized_dataset.append(anonymized_example)
        all_mappings.append(entity_mapping)

    return anonymized_dataset, all_mappings


In [ ]:
# import pickle
# with open('all_mappings1.pkl', 'wb') as fichier:
#     pickle.dump(all_mappings1, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:
# import pickle
# with open('all_mappings2.pkl', 'wb') as fichier:
#     pickle.dump(all_mappings2, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:
import pickle
with open('entity_map_qa.pkl', 'rb') as file:
    entity_mapping = pickle.load(file)
len(entity_mapping)

In [ ]:
# Assuming you have a dataset and a NER tagger already set up
anonymized_train, all_mappings1 = anonymize_dataset(dataset['train'], tagger)

 13%|█▎        | 11200/87599 [27:07<2:04:19, 10.24it/s]

2024-10-05 06:02:07,378 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-10-05 06:02:07,469 Warning: An empty Sentence was created! Are there empty strings in your dataset?


 13%|█▎        | 11202/87599 [27:07<2:02:04, 10.43it/s]

2024-10-05 06:02:07,564 Warning: An empty Sentence was created! Are there empty strings in your dataset?


 15%|█▍        | 12759/87599 [30:39<1:57:04, 10.65it/s]

...

100%|█████████▉| 87346/87599 [3:42:41<00:44,  5.70it/s]

2024-10-05 09:17:41,791 Warning: An empty Sentence was created! Are there empty strings in your dataset?


100%|█████████▉| 87347/87599 [3:42:42<00:44,  5.67it/s]

2024-10-05 09:17:41,963 Warning: An empty Sentence was created! Are there empty strings in your dataset?


100%|██████████| 87599/87599 [3:43:26<00:00,  6.53it/s]


In [ ]:
# Assuming you have a dataset and a NER tagger already set up
anonymized_val, all_mappings2 = anonymize_dataset(dataset['validation'], tagger)

  6%|▌         | 621/10570 [01:45<20:30,  8.08it/s]

2024-10-05 09:20:13,847 Warning: An empty Sentence was created! Are there empty strings in your dataset?


 21%|██▏       | 2265/10570 [06:44<26:30,  5.22it/s]

2024-10-05 09:25:12,631 Warning: An empty Sentence was created! Are there empty strings in your dataset?


 47%|████▋     | 4961/10570 [15:59<22:55,  4.08it/s]

...

 94%|█████████▍| 9987/10570 [32:55<02:34,  3.76it/s]

2024-10-05 09:51:23,636 Warning: An empty Sentence was created! Are there empty strings in your dataset?


 95%|█████████▍| 10038/10570 [33:06<01:26,  6.18it/s]

2024-10-05 09:51:34,322 Warning: An empty Sentence was created! Are there empty strings in your dataset?


100%|██████████| 10570/10570 [35:06<00:00,  5.02it/s]


In [ ]:
from datasets import Dataset

# Assuming `anonymized_train` and `anonymized_val` are lists of dictionaries,
# or each element in the list corresponds to an entry in the original dataset.

# Convert the list of anonymized examples back to a Dataset object.
anonymized_train_dataset = Dataset.from_dict({key: [d[key] for d in anonymized_train] for key in anonymized_train[0]})
anonymized_val_dataset = Dataset.from_dict({key: [d[key] for d in anonymized_val] for key in anonymized_val[0]})


In [ ]:
anonymized_train_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
del anonymized_combined

In [ ]:
anonymized_combined = DatasetDict({
    'train': anonymized_train_dataset,
    'validation': anonymized_val_dataset
})

In [ ]:
# anonymized_combined.save_to_disk("qa_good")

Saving the dataset (0/1 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk # import the load_from_disk function from the datasets library

anonymized_subset=load_from_disk("qa_good")

In [ ]:
anonymized_subset['train'][7]

{'id': '5733bf84d058e614000b61c0',
 'title': 'University_of_Notre_Dame',
 'context': 'As at most other universities, ent188895 students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the ent123800 magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in ent115227. The other magazine, ent256017, is released twice a year and focuses on student literature and artwork. The ent122511 yearbook is published annually. The newspapers have varying publication interests, with ent205802 published daily and mainly reporting university and other news, and staffed by students from both ent292193 and ent168725. Unlike ent123800 and ent41822, ent205802 is an independent publication and does not have a faculty advisor or any editorial oversight from the ent101460. In 1987, when some students believed t

In [ ]:
for i in range(20,30):
    print(dataset['train'][i])

In [ ]:
# Initialize dictionaries to keep track of mappings and used IDs
entity_mappings = defaultdict(str)
used_ids = set()

# Apply anonymization to the dataset
anonymized_subset = dataset['train'].select(range(200)).map(lambda example: anonymize_example(example, tagger,entity_mappings, used_ids))



In [ ]:
for i in range(20,41):
    print(dataset['train'][i])

{'id': '5733a70c4776f41900660f64', 'title': 'University_of_Notre_Dame', 'context': "All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exposure to any major in which they are interested. The program also includes a Learning Resource Center which provides time management, collaborative learning, and subject tutoring. This program has been recognized previously, by U.S. News & World Report, as outstanding.", 'question': 'What entity provides help with the management of time for new students at Notre Dame?', 'answers': {'text': ['Learning Resource Center'], 'answer_start': [496]}}
{'id': '5733a70c4776f41900660f62', 'title': 'Unive

In [ ]:
for i in range(20,31):
    print(anonymized_subset['train'][i])

{'id': '5733a70c4776f41900660f64', 'title': 'University_of_Notre_Dame', 'context': 'All of ent195905 undergraduate students are a part of one of the five undergraduate colleges at the school or are in the ent80919 program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exposure to any major in which they are interested. The program also includes ent209217 which provides time management, collaborative learning, and subject tutoring. This program has been recognized previously, by ent275506. ent102414, as outstanding.', 'question': 'What entity provides help with the management of time for new students at ent125700?', 'answers': {'answer_start': [478], 'text': ['ent161017']}}
{'id': '5733a70c4776f41900660f62', 'title': 'University_of_Notre_Dame', 'context': 'All of ent105906 un

In [ ]:
dataset['train'][30]

{'id': '5733ac31d058e614000b5ff6',
 'title': 'University_of_Notre_Dame',
 'context': "The Joan B. Kroc Institute for International Peace Studies at the University of Notre Dame is dedicated to research, education and outreach on the causes of violent conflict and the conditions for sustainable peace. It offers PhD, Master's, and undergraduate degrees in peace studies. It was founded in 1986 through the donations of Joan B. Kroc, the widow of McDonald's owner Ray Kroc. The institute was inspired by the vision of the Rev. Theodore M. Hesburgh CSC, President Emeritus of the University of Notre Dame. The institute has contributed to international policy discussions about peace building practices.",
 'question': "What is the title of Notre Dame's Theodore Hesburgh?",
 'answers': {'text': ['President Emeritus of the University of Notre Dame'],
  'answer_start': [466]}}

In [ ]:
anonymized_subset['train'][17]

{'id': '5733a6424776f41900660f4f',
 'title': 'University_of_Notre_Dame',
 'context': 'ent132326 was established in 1920, however, early courses in civil and mechanical engineering were a part of ent218001 since the 1870s. Today the college, housed in the ent163566, ent235446, and ent277423, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the ent202884 and ent196167 awarding additional ent47679. and ent203525 (ent146980) degrees, respectively.',
 'question': 'Before the creation of ent59705 similar studies were carried out at which ent32203 college?',
 'answers': {'answer_start': [109], 'text': ['ent218001']}}

In [ ]:
# import pickle
# with open('entity_map_qa_new.pkl', 'wb') as fichier:
#     pickle.dump(entity_mappings, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:
for i in range(0,10):
    print(anonymized_subset['train'][i])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a ent35615 character. Atop ent37396 gold dome is a golden statue of ent24885. Immediately in front of ent62154 and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to ent62154 is ent42220. Immediately behind the basilica is the Grotto, a ent78988 place of prayer and reflection. It is a replica of the grotto at ent81722, ent73597 where ent24885 reputedly appeared to ent27892 in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and ent62907), is a simple, modern stone statue of ent61792.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
{'id': '5733be284776f4190066117f', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a ent35615 character

## Anomysation

In [ ]:
anonymized_train[1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [ ]:

# Créer des objets Dataset pour chaque partie (train, validation, test)
train_dataset = Dataset.from_dict({
    'id': [example['id'] for example in anonymized_train],
    'title': [example['title'] for example in anonymized_train],
    'context': [example['context'] for example in anonymized_train],
    'question': [example['question'] for example in anonymized_train],
    'answers': [example['answers'] for example in anonymized_train]
})

val_dataset = Dataset.from_dict({
    'id': [example['id'] for example in anonymized_val],
    'title': [example['title'] for example in anonymized_val],
    'context': [example['context'] for example in anonymized_val],
    'question': [example['question'] for example in anonymized_val],
    'answers': [example['answers'] for example in anonymized_val]
})


# Combiner en un DatasetDict
final_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
})

# Afficher des informations sur le DatasetDict final
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 75608
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 8663
    })
})


In [ ]:
# import pickle

# with open('used_ids_qa.pkl', 'wb') as fichier:
#     pickle.dump(used_ids, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:

# with open('entity_map_qa.pkl', 'wb') as fichier:
#     pickle.dump(entity_mapping, fichier)

# print("Save with succes.")

Save with succes.


In [ ]:
with open('entity_map_qa.pkl', 'rb') as file:
    dictionary = pickle.load(file)
len(dictionary)

10185

In [ ]:
# final_dataset.save_to_disk("qa")

Saving the dataset (0/1 shards):   0%|          | 0/75608 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8663 [00:00<?, ? examples/s]

## Question Answering without entity

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
anonymized_subset=load_from_disk("qa_good")


In [ ]:
for i in range(0,5):
    print(anonymized_subset['train'][i])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a ent221588 character. Atop ent194266 gold dome is a golden statue of ent79924. Immediately in front of ent123323 and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to ent123323 is ent220757. Immediately behind the basilica is the Grotto, a ent108031 place of prayer and reflection. It is a replica of the grotto at ent4627, ent280607 where ent79924 reputedly appeared to ent86140 in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and ent27423), is a simple, modern stone statue of ent12688.', 'question': 'To whom did ent79924 allegedly appear in 1858 in ent4627 ent280607?', 'answers': {'answer_start': [459], 'text': ['ent86140']}}
{'id': '5733be284776f4190066117f', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a ent209755 character. Atop ent24791

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("YvanCarre/anonym-gpt2",use_fast=True)

#tokenizer = AutoTokenizer.from_pretrained("YvanCarre/gpt2_anonym",use_fast=True)


tokenizer.do_lower_case = True
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",

    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:

            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:

                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_data = anonymized_subset.map(preprocess_function, batched=True)

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10570
    })
})

In [ ]:

len(tokenizer)

32000

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments,GPT2Config
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering, Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("YvanCarre/anonym-gpt2")

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at YvanCarre/anonym-gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 768)

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10570
    })
})

In [ ]:

from transformers import Trainer, DataCollatorWithPadding
from torch.utils.data import DataLoader  #
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq,DataCollatorForLanguageModeling

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
#model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    warmup_steps=1000,
    #gradient_accumulation_steps=2,
    max_steps=10000,
    lr_scheduler_type="linear",
    logging_steps=100,
    learning_rate=3e-4,
    per_device_train_batch_size=4,  # 24 - 8
    weight_decay=0.01,
    dataloader_num_workers=4,
    save_steps=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    fp16=True,
    #report_to="tensorboard",
    eval_steps=0.1,
    seed=42,
    #remove_unused_columns=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: carre (carre-su). Use `wandb login --relogin` to force relogin


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
1000,2.632700,2.512508
2000,2.283600,2.189015
3000,2.066300,2.101694
4000,2.054400,2.006934
5000,2.039000,1.944141


wandb: Network error resolved after 0:01:28.187385, resuming normal operation.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code

In [ ]:
trainer.evaluate()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.7574889659881592,
 'eval_runtime': 190.4695,
 'eval_samples_per_second': 55.494,
 'eval_steps_per_second': 3.47,
 'epoch': 0.639269406392694}

## Test

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering
import torch

def generate_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=384, truncation=True, padding="max_length")

    ## Move input tensors to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Add this line

    with torch.no_grad():
        outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    ## Use the 'decode' method directly on the tokenizer
    answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])
    return answer

In [ ]:
context = "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France and one of the most recognizable structures in the world."

questions = [
    "Who designed the Eiffel Tower?",
    "When was the Eiffel Tower built?",
    "Where is the Eiffel Tower located?",
    "What was the initial reaction to the Eiffel Tower?",
    "What is the Eiffel Tower made of?"
]


for question in questions:
    answer = generate_answer(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: Who designed the Eiffel Tower?
Answer:  Gustave E

Question: When was the Eiffel Tower built?
Answer: 1887 to 1889

Question: Where is the Eiffel Tower located?
Answer:  Champ de Mars

Question: What was the initial reaction to the Eiffel Tower?
Answer:  criticized by some of France's leading artists and intellectuals for its design

Question: What is the Eiffel Tower made of?
Answer:  wrought-iron lattice tower



In [ ]:
idx=85

test_question = anonymized_subset['validation'][idx]['question']
test_context = anonymized_subset['validation'][idx]['context']
generated_answer = generate_answer(test_question, test_context)
print(f"Generated answer: {generated_answer}")


Generated answer: ent54956


In [ ]:
anonymized_subset['validation'][idx]

{'id': '56beaf5e3aeaaa14008c91fd',
 'title': 'Super_Bowl_50',
 'context': 'ent54956 broadcast ent136160 in the ent249541, and charged an average of $5 million for a 30-second commercial during the game. The ent136160 halftime show was headlined by the ent116144 rock group ent217509 with special guest performers ent130408 and ent114873, who headlined the ent286281 and ent299233 halftime shows, respectively. It was the third-most watched ent254499. broadcast ever.',
 'question': 'Who was the broadcaster for ent136160 in ent185589?',
 'answers': {'answer_start': [0, 0, 0],
  'text': ['ent54956', 'ent54956', 'ent54956']}}

## Evaluate

In [ ]:
!rm -rf ./results

In [ ]:

# Reverse the dictionary to create a lookup for deanonymization
def deanonymize_list(text, data):
    reverse_data = {v: k for k, v in data.items()}

    # Split the input text into words, preserving punctuation
    words = text.split()

    # Replace identifiers with corresponding keys, preserving punctuation
    deanonymized_words = []
    for word in words:
        # Remove punctuation for lookup but keep it for the output
        stripped_word = word.strip(',.')

        # Get the deanonymized word, or keep the original word if not found
        deanonymized_word = reverse_data.get(stripped_word, stripped_word)

        # Reattach any stripped punctuation
        if stripped_word != deanonymized_word:  # It was an entity that was replaced
            deanonymized_word += word[len(stripped_word):]  # Add original punctuation

        deanonymized_words.append(deanonymized_word)

    # Join the replaced words back into a single string
    return ' '.join(deanonymized_words)


def deanonymize_entity_string(entity_string, data):
    reverse_data = {v: k for k, v in data.items()}

    # Split the input string by spaces to handle identifiers
    words = entity_string.split()

    # Replace identifiers with corresponding keys, preserving punctuation
    deanonymized_words = []
    for word in words:
        # Remove punctuation for lookup but keep it for the output
        stripped_word = word.strip(',.')

        # Get the deanonymized word or keep the original if not found
        deanonymized_word = reverse_data.get(stripped_word, stripped_word)

        # Reattach any stripped punctuation
        if stripped_word != deanonymized_word:  # It was an entity that was replaced
            deanonymized_word += word[len(stripped_word):]  # Add original punctuation

        deanonymized_words.append(deanonymized_word)

    # Join the replaced words back into a single string
    return ' '.join(deanonymized_words)

In [ ]:
import pickle
with open('all_mappings2.pkl', 'rb') as file:
    entity_mapping = pickle.load(file)
len(entity_mapping)

10570

In [ ]:
from tqdm import tqdm


def evaluate_model_with_rouge(model, tokenizer, dataset):
    exact_match_total = 0
    f1_total = 0

    num_samples = len(dataset['validation'])
    for i in tqdm(range(num_samples)):
        question = dataset['validation'][i]['question']
        context = dataset['validation'][i]['context']
        ground_truths = dataset['validation'][i]['answers']['text']

        generated_answer = generate_answer(question, context)

        exact_match_scores = [int(deanonymize_entity_string(generated_answer,entity_mapping[i]).strip() == deanonymize_entity_string(truth,entity_mapping[i]).strip()) for truth in ground_truths]
        exact_match_total += max(exact_match_scores)

        f1_scores = []

        for truth in ground_truths:
            generated_tokens = generated_answer.split()
            ground_truth_tokens = truth.split()

            common_tokens = set(generated_tokens) & set(ground_truth_tokens)
            precision = len(common_tokens) / len(generated_tokens) if generated_tokens else 0
            recall = len(common_tokens) / len(ground_truth_tokens) if ground_truth_tokens else 0
            f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            f1_scores.append(f1)

        f1_total += max(f1_scores)

    avg_exact_match = exact_match_total / num_samples
    avg_f1_score = f1_total / num_samples

    return avg_exact_match, avg_f1_score

In [ ]:
avg_exact_match, avg_f1_score= evaluate_model_with_rouge(model, tokenizer, anonymized_subset)

100%|██████████| 10570/10570 [09:42<00:00, 18.14it/s]


In [ ]:
print(f"Exact Match: {avg_exact_match:.2f}")
print(f"F1 Score: {avg_f1_score:.2f}")

Exact Match: 0.54
F1 Score: 0.61


In [ ]:
print(f"Exact Match: {avg_exact_match:.2f}")
print(f"F1 Score: {avg_f1_score:.2f}")

Exact Match: 0.54
F1 Score: 0.61


In [ ]:
!rm -rf ./results

In [ ]:
avg_exact_match, avg_f1_score, avg_rouge_1, avg_rouge_2, avg_rouge_l = evaluate_model_with_rouge(model, tokenizer, anonymized_subset)
print(f"Exact Match: {avg_exact_match:.2f}")
print(f"F1 Score: {avg_f1_score:.2f}")
print(f"ROUGE-1: {avg_rouge_1:.2f}")
print(f"ROUGE-2: {avg_rouge_2:.2f}")
print(f"ROUGE-L: {avg_rouge_l:.2f}")

100%|██████████| 8663/8663 [05:07<00:00, 28.21it/s]

Exact Match: 0.55
F1 Score: 0.66
ROUGE-1: 0.67
ROUGE-2: 0.35
ROUGE-L: 0.67
